In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mbee import compute_edits_and_insertions
from insert_convert import InsertionConverter, get_number_of_masks
from trying import PointingConverter

In [3]:
import pandas as pd

In [4]:
max_mask = 4
use_pointing = True

In [5]:
label_map = {'PAD': 0, 'SWAP': 1, 'KEEP': 2, 'DELETE': 3}
# Create Insert 1 MASK to insertion N MASKS.
for i in range(1, max_mask+1):
    label_map[f'KEEP|{i}'] = len(label_map)
if not use_pointing:
    label_map[f'DELETE|{i}'] = len(label_map)

In [6]:
label_map

{'PAD': 0,
 'SWAP': 1,
 'KEEP': 2,
 'DELETE': 3,
 'KEEP|1': 4,
 'KEEP|2': 5,
 'KEEP|3': 6,
 'KEEP|4': 7}

In [7]:
compute_edits_and_insertions(['haryo', 'makan', 'jokowi'], ['haryo',  'nasi', 'jokowi', 'goreng', 'makan', 'enak'], 5)

(['[KEEP]', '[KEEP]', '[DELETE]'],
 [['nasi', 'jokowi', 'goreng'], [], ['enak']])

In [8]:
point_converter = PointingConverter({}, False)

In [9]:
points = point_converter.compute_points(['[CLS]', 'haryo', 'makan', '[SEP]'], ' '.join(['[CLS]', 'haryo',  'nasi', 'jokowi', 'goreng', 'makan', 'enak','[SEP]']))

In [10]:
points

[1|, 2|nasi jokowi goreng, 3|enak, 0|]

In [11]:
points = point_converter.compute_points(
    ["CLS", "a", "b", "a", "##d", "##e", "[SEP]"],
    " ".join(["[CLS]", "a", "a", "##d", "##e", "[SEP]"]),
)

In [12]:
points

[1|, 3|, 0|, 4|, 5|, 6|, 0|]

In [13]:
labels = [t.added_phrase for t in points]
point_indexes = [t.point_index for t in points]
point_indexes_set = set(point_indexes)
new_labels = []
for i, added_phrase in enumerate(labels):
    if i not in point_indexes_set:
        new_labels.append(label_map['DELETE'])
    elif not added_phrase:
        new_labels.append(label_map['KEEP'])
    else:
        new_labels.append(label_map['KEEP|' +
                                            str(len(added_phrase.split()))])
labels = new_labels

In [14]:
point_indexes_set

{0, 1, 3, 4, 5, 6}

In [15]:
labels

[2, 2, 3, 2, 2, 2, 2]

In [16]:
insertion_converter = InsertionConverter(20, label_map, tokenized_data, use_pointing)

NameError: name 'tokenized_data' is not defined

In [17]:
from transformers import AutoTokenizer, AutoModel

/home/haryoaw/mambaforge/envs/sensei/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AutoTokenizer.from_pretrained('bert-base-cased')

In [18]:
tokenizer_bert = AutoTokenizer.from_pretrained('indolem/indobert-base-uncased')

In [19]:
def create_label_dict(max_mask=3, use_pointing=True):
    label_map = {'PAD': 0, 'SWAP': 1, 'KEEP': 2, 'DELETE': 3}
    # Create Insert 1 MASK to insertion N MASKS.
    for i in range(1, max_mask+1):
        label_map[f'KEEP|{i}'] = len(label_map)
    if not use_pointing:
        label_map[f'DELETE|{i}'] = len(label_map)
    return label_map

In [20]:
label_dict = create_label_dict()

In [21]:
import pandas as pd

In [22]:
df_train = pd.read_csv("../data/stif-indonesia/train.csv")

In [23]:
example_instance = df_train.iloc[0]

In [24]:
mbek = compute_edits_and_insertions(example_instance.informal, example_instance.formal, 5)

In [66]:
informal_instance = tokenizer_bert.tokenize(example_instance.informal, add_special_tokens=True)
formal_instance = tokenizer_bert.tokenize(example_instance.formal, add_special_tokens=True)

In [26]:
point_converter = PointingConverter({}, False)

In [27]:
points = point_converter.compute_points(informal_instance, ' '.join(formal_instance))

In [28]:
def create_pointer_labels(points, label_map):
    labels = [t.added_phrase for t in points]
    point_indexes = [t.point_index for t in points]
    point_indexes_set = set(point_indexes)
    new_labels = []
    for i, added_phrase in enumerate(labels):
        if i not in point_indexes_set:
            new_labels.append(label_map["DELETE"])
        elif not added_phrase:
            new_labels.append(label_map["KEEP"])
        else:
            new_labels.append(label_map["KEEP|" + str(len(added_phrase.split()))])
    return new_labels

In [29]:
label_dict

{'PAD': 0,
 'SWAP': 1,
 'KEEP': 2,
 'DELETE': 3,
 'KEEP|1': 4,
 'KEEP|2': 5,
 'KEEP|3': 6}

In [30]:
label = create_pointer_labels(points, label_dict)

In [31]:
point_indexes = [t.point_index for t in points]

In [32]:
def get_pointer_and_label(x, label_dict, point_converter):
    informal_instance = tokenizer_bert.tokenize(x.informal, add_special_tokens=True)
    formal_instance = tokenizer_bert.tokenize(x.formal, add_special_tokens=True)
    points = point_converter.compute_points(informal_instance, ' '.join(formal_instance))
    label = create_pointer_labels(points, label_dict)
    point_indexes = [t.point_index for t in points]
    return point_indexes, label
    

In [33]:
label_dict = create_label_dict(30)


In [34]:
label_dict

{'PAD': 0,
 'SWAP': 1,
 'KEEP': 2,
 'DELETE': 3,
 'KEEP|1': 4,
 'KEEP|2': 5,
 'KEEP|3': 6,
 'KEEP|4': 7,
 'KEEP|5': 8,
 'KEEP|6': 9,
 'KEEP|7': 10,
 'KEEP|8': 11,
 'KEEP|9': 12,
 'KEEP|10': 13,
 'KEEP|11': 14,
 'KEEP|12': 15,
 'KEEP|13': 16,
 'KEEP|14': 17,
 'KEEP|15': 18,
 'KEEP|16': 19,
 'KEEP|17': 20,
 'KEEP|18': 21,
 'KEEP|19': 22,
 'KEEP|20': 23,
 'KEEP|21': 24,
 'KEEP|22': 25,
 'KEEP|23': 26,
 'KEEP|24': 27,
 'KEEP|25': 28,
 'KEEP|26': 29,
 'KEEP|27': 30,
 'KEEP|28': 31,
 'KEEP|29': 32,
 'KEEP|30': 33}

In [35]:
a = df_train.apply(
    lambda x: get_pointer_and_label(x, label_dict, point_converter), axis=1
)
# unpack to two series
point_indexes, label = zip(*a)

# append to dataframe
df_train['point_indexes'] = point_indexes
df_train['label'] = label
df_train.to_csv("train_with_pointing.csv", index=False)


In [36]:
# do it for validation and test too
df_val = pd.read_csv("../data/stif-indonesia/dev.csv")
df_test = pd.read_csv("../data/stif-indonesia/test.csv")

a = df_val.apply(
    lambda x: get_pointer_and_label(x, label_dict, point_converter), axis=1
)
# unpack to two series
point_indexes, label = zip(*a)
df_val['point_indexes'] = point_indexes
df_val['label'] = label

df_val.to_csv("dev_with_pointing.csv", index=False)

a = df_test.apply(
    lambda x: get_pointer_and_label(x, label_dict, point_converter), axis=1
)
# unpack to two series
point_indexes, label = zip(*a)
df_test['point_indexes'] = point_indexes
df_test['label'] = label

df_test.to_csv("test_with_pointing.csv", index=False)

In [37]:
x = df_train.label.tolist()

# flatten x
x = [item for sublist in x for item in sublist]


In [38]:
# count x
from collections import Counter
Counter(x)

Counter({2: 22550,
         3: 12954,
         4: 4069,
         5: 1386,
         6: 670,
         7: 290,
         8: 124,
         9: 68,
         10: 29,
         11: 15,
         12: 5,
         13: 4,
         14: 3,
         17: 1,
         15: 1,
         16: 1})

In [39]:
# calculate len describe label df_train
df_train['len_label'] = df_train.label.apply(lambda x: len(x))
df_train.len_label.describe()

count    1922.000000
mean       21.940687
std        10.131238
min         7.000000
25%        14.000000
50%        20.000000
75%        28.000000
max       100.000000
Name: len_label, dtype: float64

In [40]:
pd.set_option('max_colwidth', 0)

In [41]:
df_train[df_train.len_label > 90]

,informal,formal,point_indexes,label,len_label
780,"pagi admin , paket saya dari shopee dengan kode xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx r04b tidak kebaca nomor resinya , mohon dibantu ya . dari hari jumat masih ngendep di jne malang",pagi admin . paket saya dari shopee dengan kode xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx xxxnumberxxx r04b tidak terbaca nomor resinya . mohon dibantu . dari hari jumat masih mengendap di jne malang .,"[1, 2, 80, 0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 73, 0, 0, 74, 75, 77, 0, 78, 81, 0, 4, 82, 83, 84, 88, 0, 0, 0, 89, 90, 91, 92, 0]","[2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 3, 2, 2, 4, 3, 2, 4, 3, 2, 2, 2, 2, 4, 3, 3, 3, 2, 2, 2, 4, 2]",93
1220,"sudah , harga barang xxxnumberxxx xxxnumberxxx xxxnumberxxx k , ongkir xxxnumberxxx xxxnumberxxx k , diskon xxxnumberxxx xxxnumberxxx k , biaya penanganan xxxnumberxxx , xxxnumberxxx k . seharusnya saya bayar xxxnumberxxx , xxxnumberxxx k saja , tapi malah ditagih xxxnumberxxx , xxxnumberxxx k .","sudah , harga barang xxxnumberxxx ribu , onkir xxxnumberxxx ribu , diskon xxxnumberxxx ribu , biaya penanganan xxxnumberxxx ribu . seharusnya saya bayar xxxnumberxxx ribu saja , tapi malah ditagih xxxnumberxxx ribu .","[1, 2, 3, 4, 5, 6, 7, 8, 9, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 0, 24, 25, 26, 27, 28, 35, 0, 0, 0, 0, 0, 0, 36, 37, 38, 39, 40, 41, 48, 0, 0, 0, 0, 0, 0, 49, 50, 51, 52, 53, 54, 55, 63, 0, 0, 0, 0, 0, 0, 0, 64, 65, 66, 67, 68, 69, 70, 71, 79, 0, 0, 0, 0, 0, 0, 0, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 98, 0, 0, 0, 0, 0, 0, 0, 99, 0]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2]",100


In [42]:
df_train.to_csv("train_with_pointing.csv", index=False)

In [45]:
# input: input_tokens, labels, point_indexes, output_tokens
# input_tokens: tokenized input
# labels: label (MASK|xxx, delete, keep, etc)
# point_indexes: index of the token that is pointed to
# output_tokens: tokenized target

informal_instance

['[CLS]',
 'alhamdulillah',
 'st',
 '##l',
 '##h',
 'libur',
 'xxx',
 '##num',
 '##ber',
 '##xx',
 '##x',
 'hari',
 'on',
 '##bi',
 '##d',
 'lg',
 '##sg',
 'dikasih',
 'order',
 '##an',
 ',',
 'food',
 'lg',
 '.',
 'thanks',
 'xxx',
 '##user',
 '##xx',
 '##x',
 'cc',
 '[SEP]']

In [49]:
tokenizer_bert.vocab['[PAD]']

KeyError: '[UNUSED]'

In [72]:
def create_masked_source(source_tokens, labels, source_indexes,
                        target_tokens):
    """Realizes source_tokens & adds deleted to source_tokens and target_tokens.

    Args:
        source_tokens: List of source tokens.
        labels: List of label IDs, which correspond to a list of labels (KEEP,
        DELETE, MASK|1, MASK|2...).
        source_indexes: List of next tokens (see pointing converter for more
        details) (ordered by source tokens)
        target_tokens: Optional list of target tokens. Only provided when
        constructing training examples.

    Returns:
        masked_tokens: The source input for the insertion model, including MASK
        tokens and bracketed deleted tokens.
        target_tokens: The target tokens for the insertion model, where mask
        tokens are replaced with the actual token, also includes bracketed
        deleted tokens.
    """
    DELETE_SPAN_START = '[UNK]'
    DELETE_SPAN_END = '[PAD]'
    current_index = 0
    masked_tokens = []

    label_map_inverse = {v: k for k, v in label_map.items()}

    kept_tokens = set([0])
    for _ in range(len(source_tokens)):
        current_index = source_indexes[current_index]
        kept_tokens.add(current_index)
        # Token is deleted.
        if current_index == 0:
            break

    current_index = 0
    for _ in range(len(source_tokens)):
        source_token = source_tokens[current_index]
        deleted_tokens = []
        # Looking forward finding all deleted tokens.
        for i in range(current_index + 1, len(source_tokens)):
        ## If not a deleted token.
            if i in kept_tokens:
                break
            deleted_tokens.append(source_tokens[i])

        # Add deleted tokens to masked_tokens and target_tokens.
        masked_tokens.append(source_token)
        # number_of_masks specifies the number MASKED tokens which
        # are added to masked_tokens.
        number_of_masks = get_number_of_masks(
            label_map_inverse[labels[current_index]])
        for _ in range(number_of_masks):
            masked_tokens.append('[MASK]')
        if deleted_tokens:
            masked_tokens_length = len(masked_tokens)
            bracketed_deleted_tokens = ([DELETE_SPAN_START] +
                                    deleted_tokens +
                                        [DELETE_SPAN_END])
            target_tokens = (
                target_tokens[:masked_tokens_length] + bracketed_deleted_tokens +
                target_tokens[masked_tokens_length:])
            masked_tokens += bracketed_deleted_tokens

        current_index = source_indexes[current_index]
        if current_index == 0:
            break
    return masked_tokens, target_tokens

In [73]:
example_instance = df_train.iloc[0]
first_label = df_train.label.tolist()[0]
first_point_indexes = df_train.point_indexes.tolist()[0]
informal_instance = tokenizer_bert.tokenize(example_instance.informal, add_special_tokens=True)
formal_instance = tokenizer_bert.tokenize(example_instance.formal, add_special_tokens=True)

In [94]:
tokenizer_bert.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [95]:
print(' '.join(informal_instance))
print(' '.join(formal_instance))

[CLS] alhamdulillah st ##l ##h libur xxx ##num ##ber ##xx ##x hari on ##bi ##d lg ##sg dikasih order ##an , food lg . thanks xxx ##user ##xx ##x cc [SEP]
[CLS] alhamdulillah setelah libur xxx ##num ##ber ##xx ##x hari on ##bi ##d langsung diberi order , makanan lagi . terima kasih xxx ##user ##xx ##x cc . [SEP]


In [84]:
masked_tokens, target_tokens = create_masked_source(informal_instance, 
                     first_label, 
                     first_point_indexes, 
                     formal_instance)

In [87]:
import random

In [88]:
if target_tokens and '[MASK]' not in masked_tokens:
      # Generate random MASKs.
    # Don't mask the start or end token.
    indexes = list(range(1, len(masked_tokens) - 1))
    random.shuffle(indexes)
    # Limit MASK to ~10% of the source tokens.
    indexes = indexes[:int(len(masked_tokens) * 0.1)]
    for index in indexes:
        masked_tokens[index] = '[MASK]'

In [90]:
len(masked_tokens)

49

In [93]:
list(zip(masked_tokens, target_tokens))

[('[CLS]', '[CLS]'),
 ('alhamdulillah', 'alhamdulillah'),
 ('[MASK]', 'setelah'),
 ('[unused1]', '[unused1]'),
 ('st', 'st'),
 ('##l', '##l'),
 ('##h', '##h'),
 ('[unused2]', '[unused2]'),
 ('libur', 'libur'),
 ('xxx', 'xxx'),
 ('##num', '##num'),
 ('##ber', '##ber'),
 ('##xx', '##xx'),
 ('##x', '##x'),
 ('hari', 'hari'),
 ('on', 'on'),
 ('##bi', '##bi'),
 ('##d', '##d'),
 ('[MASK]', 'langsung'),
 ('[MASK]', 'diberi'),
 ('[unused1]', '[unused1]'),
 ('lg', 'lg'),
 ('##sg', '##sg'),
 ('dikasih', 'dikasih'),
 ('[unused2]', '[unused2]'),
 ('order', 'order'),
 ('[unused1]', '[unused1]'),
 ('##an', '##an'),
 ('[unused2]', '[unused2]'),
 (',', ','),
 ('[MASK]', 'makanan'),
 ('[MASK]', 'lagi'),
 ('[unused1]', '[unused1]'),
 ('food', 'food'),
 ('lg', 'lg'),
 ('[unused2]', '[unused2]'),
 ('.', '.'),
 ('[MASK]', 'terima'),
 ('[MASK]', 'kasih'),
 ('[unused1]', '[unused1]'),
 ('thanks', 'thanks'),
 ('[unused2]', '[unused2]'),
 ('xxx', 'xxx'),
 ('##user', '##user'),
 ('##xx', '##xx'),
 ('##x', '##x'

In [91]:
len(target_tokens)

49